<a href="https://colab.research.google.com/github/sadrabr/LLM_projects/blob/main/Code_generator_frontier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q openai
!pip install -q google-generativeai
!pip install -q anthropic
!pip install -q gradio

In [30]:
# imports

import os
import io
import sys
from openai import OpenAI
import google.generativeai as genai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [31]:
# Sign in to frontier model using Secrets in Colab

from google.colab import userdata

openai_api_key = userdata.get('OPEN_AI')
openai = OpenAI(api_key=openai_api_key)

anthropic_api_key = userdata.get('ANTHROPIC_AI')
claude = anthropic.Anthropic(api_key=anthropic_api_key)

google_api_key = userdata.get('GEMINI_API')
gemini = genai.configure(api_key=google_api_key)

In [52]:
# Iitialize

OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL="gemini-2.5-flash-preview-04-17"

In [33]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an T4 GPU on google colab. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [34]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [35]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [62]:
def convert_messages_to_prompt(messages):
    return "\n\n".join(f"{m['role'].capitalize()}: {m['content']}" for m in messages)

In [36]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [37]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [38]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [71]:
def stream_gemini(python):
    model = genai.GenerativeModel(GEMINI_MODEL)
    prompt = convert_messages_to_prompt(messages_for(python))

    try:
        response = model.generate_content(prompt, stream=True)

        reply = ""
        for chunk in response:
            if hasattr(chunk, 'text') and chunk.text:
                print(chunk.text, end='', flush=True)
                reply += chunk.text

        if not reply.strip():
            print("\n❌ No content received from Gemini.")
        else:
            write_output(reply)

    except Exception as e:
        print(f"\n⚠️ Error during generation: {e}")

In [40]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [41]:
exec(pi)

In [72]:
stream_gemini(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform the calculation
double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        // Use long long for j to prevent overflow with large i * param1
        long long j1 = i * param1 - param2;
        result -= (1.0 / j1); // Perform floating-point division

        long long j2 = i * param1 + param2;
        result += (1.0 / j2); // Perform floating-point division
    }
    return result;
}

int main() {
    // Use long long for iterations as per Python example (100 million)
    long long iterations = 100000000;
    int param1 = 4;
    int param2 = 1;

    // Record start time
    auto start_time = std::chrono::high_resolution_clock::now();

    // Perform the calculation
    double result = calculate(iterations, param1, param2) * 4;

    // Record end time
    auto end_time = std::chrono::high_resolution_clock::now();


In [66]:
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.303772 seconds


In [75]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="GEMINI":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","GEMINI"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [77]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [78]:
import subprocess

def execute_cpp(code):
    write_output(code)
    try:
        compile_cmd = [
            "clang++",
            "-Ofast",
            "-std=c++17",
            "-march=native",  # تطبیق با معماری x86_64 سیستم Colab
            "-o", "optimized",
            "optimized.cpp"
        ]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)

        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)

        return run_result.stdout

    except subprocess.CalledProcessError as e:
        return f"❌ An error occurred:\n{e.stderr}"


In [79]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [81]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","GEMINI"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://870692fba2e164a0eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
